# Setup

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
plt.style.use('dark_background')
import sys
sys.path.insert(1, '/home/mauricio/code/mcr')
from mcr.util import glimpse, plot_value_counts, plot_value_counts_timeseries, missing_report, plot_missing, plot_unique, plot_duplicates, size

from pyspark import SparkContext
# SparkContext.getOrCreate(conf: Optional[pyspark.conf.SparkConf] = None) -> 'SparkContext'
sc = SparkContext.getOrCreate()

# sc.setLogLevel('DEBUG')

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('spark_application').getOrCreate()
print(spark.version)

from pyspark.sql import functions as F
from pyspark.sql.types import *

23/07/05 19:00:29 WARN Utils: Your hostname, rig resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface enp6s0)
23/07/05 19:00:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 19:00:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.4.1


# Introduction

## Exercises

### Loading flights data

In [2]:
# Read data from CSV file
flights = spark.read.csv('flights-larger.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
print(flights.dtypes)

The data contain 275000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| null|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

[('mon', 'int'), ('dom', 'int'), ('dow', 'int'), ('carrier', 'string'), ('flight', 'int'), ('org', 'string'), ('mile', 'int'), ('depart', 'double'), ('duration', 'int'), ('delay', 'int')]


### Loading SMS spam data

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

# Load data from a delimited file
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema)

# Print schema of DataFrame
sms.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



# Classification

## Data preparation

### Exercises

#### Removing columns and rows

In [4]:
print(f'{flights.count()=}')
# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Number of records with missing 'delay' values
print(f"{flights_drop_column.filter('delay IS NULL').count()=}")

# Remove records with missing 'delay' values
flights_valid_delay = flights_drop_column.filter('delay IS NOT NULL')
print(f'{flights_valid_delay.count()=}')

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights_valid_delay.dropna()
print(f'{flights_none_missing.count()=}')

flights.count()=275000
flights_drop_column.filter('delay IS NULL').count()=16711
flights_valid_delay.count()=258289
flights_none_missing.count()=258289


#### Column manipulation

In [5]:
# Import the required function
# Convert 'mile' to 'km' and drop 'mile' column (1 mile is equivalent to 1.60934 km)
flights_km = flights.withColumn('km', F.round(F.col('mile') * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (F.col('delay') >= 15).cast('integer'))

# Check first five records
flights_km.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
| 10| 10|  1|     OO|  5836|ORD|  8.18|      51|   27| 253.0|    1|
|  1|  4|  1|     OO|  5866|ORD|  15.5|     102| null| 750.0| null|
| 11| 22|  1|     OO|  6016|ORD|  7.17|     127|  -19|1188.0|    0|
|  2| 14|  5|     B6|   199|JFK| 21.17|     365|   60|3618.0|    1|
|  5| 25|  3|     WN|  1675|SJC| 12.92|      85|   22| 621.0|    1|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
only showing top 5 rows



#### Categorical columns

In [6]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)
flights_indexed.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+-----------+-------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|carrier_idx|org_idx|
+---+---+---+-------+------+---+----+------+--------+-----+-----------+-------+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|        2.0|    0.0|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| null|        2.0|    0.0|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|        2.0|    0.0|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|        4.0|    2.0|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|        3.0|    5.0|
+---+---+---+-------+------+---+----+------+--------+-----+-----------+-------+
only showing top 5 rows



#### Assembling columns

In [7]:
# reproducing previous transformations
flights = flights\
    .drop('flight')\
    .filter('delay IS NOT NULL')\
    .dropna()\
    .withColumn('km', F.round(F.col('mile') * 1.60934, 0))\
    .drop('mile')\
    .withColumn('label', (F.col('delay') >= 15).cast('integer'))

flights = StringIndexer(inputCol='carrier', outputCol='carrier_idx')\
    .fit(flights)\
    .transform(flights)

flights = StringIndexer(inputCol='org', outputCol='org_idx')\
    .fit(flights)\
    .transform(flights)

In [8]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon',
    'dom',
    'dow',
    'carrier_idx',
    'org_idx',
    'km',
    'depart',
    'duration'
], outputCol='features')

# Consolidate predictor columns
flights = assembler.transform(flights)

# Check the resulting column
flights.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[10.0,10.0,1.0,2.0,0.0,253.0,8.18,51.0]  |27   |
|[11.0,22.0,1.0,2.0,0.0,1188.0,7.17,127.0]|-19  |
|[2.0,14.0,5.0,4.0,2.0,3618.0,21.17,365.0]|60   |
|[5.0,25.0,3.0,3.0,5.0,621.0,12.92,85.0]  |22   |
|[3.0,28.0,1.0,4.0,3.0,1732.0,13.33,182.0]|70   |
+-----------------------------------------+-----+
only showing top 5 rows



## Decision Tree

### Exercises

#### Train/test split

In [9]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)

# Check that training set has around 80% of records
flights_train.count() / flights.count()

0.8007387074168857

#### Build a Decision Tree

In [10]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|1    |0.0       |[0.6224005522478212,0.3775994477521788]|
|0    |1.0       |[0.3236222942591237,0.6763777057408763]|
|1    |0.0       |[0.6224005522478212,0.3775994477521788]|
|0    |1.0       |[0.3236222942591237,0.6763777057408763]|
|0    |1.0       |[0.3236222942591237,0.6763777057408763]|
+-----+----------+---------------------------------------+
only showing top 5 rows



#### Evaluate the Decision Tree

In [11]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 9934|
|    0|       0.0|16558|
|    1|       1.0|16210|
|    0|       1.0| 8765|
+-----+----------+-----+



0.6366798142499077


## Logistic Regression

### Precision and recall

$$Precision = \frac{TP}{TP + FP}$$

$$Recall = \frac{TP}{TP + FN}$$

### Weighted metrics
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    evaluator = MulticlassClassificationEvaluator()
    evaluator.evaluate(prediction, {evaluator.metricName: 'weightedPrecision'})

Other metrics:
* weightedRecall
* accuracy
* f1 - the harmonic mean of precision and recall, which is generally more robust than the accuracy.

### ROC and AUC
ROC = "Receiver Operating Characteristic"
* TP versus FP
* threshold = 0 (top right)
* threshold = 1 (bottom left)

AUC = "Area under the curve"
* ideally AUC = 1

### Exercises

#### Build a Logistic Regression model

In [12]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 9499|
|    0|       0.0|14779|
|    1|       1.0|16645|
|    0|       1.0|10544|
+-----+----------+-----+



#### Evaluate the Logistic Regression model

In [13]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP+TN) / (TP+FP+TN+FN)
print('precision = {:.2f}\nrecall    = {:.2f}\naccuracy  = {:.2f}'.format(precision, recall, accuracy))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: 'areaUnderROC'})

precision = 0.61
recall    = 0.64
accuracy  = 0.61


> The weighted precision indicates what proportion of predictions (positive and negative) are correct.

## Turn Text into Tables

### Punctuation, numbers and tokens

Removing punctuation

        from pyspark.sql.functions import regexp_replace
        # Regular expression (REGEX) to match commas and hyphens
        REGEX = '[,\\-]'
        books = books.withColumn('text', regexp_replace(books.text, REGEX, ' '))

Text to tokens

    from pyspark.ml.feature import Tokenizer
    books = Tokenizer(inputCol="text", outputCol="tokens").transform(books)

### Stop words, hashing and common words (IDF)

What are stop words?

    from pyspark.ml.feature import StopWordsRemover
    stopwords = StopWordsRemover()
    # Take a look at the list of stop words
    stopwords.getStopWords()

Removing stop words

    # Specify the input and output column names
    stopwords = stopwords.setInputCol('tokens').setOutputCol('words')
    books = stopwords.transform(books)
    
Feature hashing

    from pyspark.ml.feature import HashingTF
    hasher = HashingTF(inputCol="words", outputCol="hash", numFeatures=32)
    books = hasher.transform(books)

Dealing with common words

    from pyspark.ml.feature import IDF
    books = IDF(inputCol="hash", outputCol="features").fit(books).transform(books)

### Exercises

#### Punctuation, numbers and tokens

In [15]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '\\d+', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



#### Stop words and hashing

In [16]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms')\
      .transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024)\
      .transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,384,577,996],[2.273418200008753,3.6288353225642043,3.5890949939146903,4.104259019279279])|
|[dont, worry, guess, busy]      |(1024,[215,233,276,329],[3.9913186080986836,3.3790235241678332,4.734227298217693,4.58299632849377]) |
|[call, freephone]               |(1024,[133,138],[5.367951058306837,2.273418200008753])                                              |
|[win, cash, prize, prize, worth]|(1024,[31,47,62,389],[3.6632029660684124,4.754846585420428,4.072170704727778,7.064594791043114])    |
+--------------------------------+--------------

#### Train a spam classifer

In [17]:
sms = tf_idf.select('label', 'features')
# Split the data into training and testing sets
sms_train, sms_test = sms.randomSplit([.8, .2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   39|
|    0|       0.0|  932|
|    1|       1.0|  121|
|    0|       1.0|    4|
+-----+----------+-----+



In [18]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP+TN) / (TP+FP+TN+FN)
print('precision = {:.2f}\nrecall    = {:.2f}\naccuracy  = {:.2f}'.format(precision, recall, accuracy))

precision = 0.97
recall    = 0.76
accuracy  = 0.96


# Regression

## One-Hot Encoding

### Dense versus sparse

In [19]:
from pyspark.ml.linalg import DenseVector, SparseVector

Store this vector as Dense: [1, 0, 0, 0, 0, 7, 0, 0].

In [20]:
DenseVector([1, 0, 0, 0, 0, 7, 0, 0])

DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 0.0])

Store same vector as Sparse

In [21]:
SparseVector(8, [0, 5], [1, 7])

SparseVector(8, {0: 1.0, 5: 7.0})

In [22]:
DenseVector([1, 0, 0, 0, 0, 7, 0, 0]) == SparseVector(8, [0, 5], [1, 7])

True

> **Sparse representation is essential for effective one-hot encoding on large data sets.**

### Exercises

#### Encoding flight origin

In [23]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoder

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights_onehot = onehot.transform(flights)

# Check the results
flights_onehot.groupBy('org', 'org_idx', 'org_dummy').count().orderBy('count', ascending=False).show()

+---+-------+-------------+-----+
|org|org_idx|    org_dummy|count|
+---+-------+-------------+-----+
|ORD|    0.0|(7,[0],[1.0])|98251|
|SFO|    1.0|(7,[1],[1.0])|50011|
|JFK|    2.0|(7,[2],[1.0])|41068|
|LGA|    3.0|(7,[3],[1.0])|25180|
|SMF|    4.0|(7,[4],[1.0])|16381|
|SJC|    5.0|(7,[5],[1.0])|16068|
|TUS|    6.0|(7,[6],[1.0])| 6025|
|OGG|    7.0|    (7,[],[])| 5305|
+---+-------+-------------+-----+



## Regression

### Calculate RMSE
        from pyspark.ml.evaluation import RegressionEvaluator
        # Find RMSE (Root Mean Squared Error)
        RegressionEvaluator(labelCol='consumption').evaluate(predictions)
        0.708699086182001
A RegressionEvaluator can also calculate the following metrics:
* mae (Mean Absolute Error)
* r2 (R )
* mse (Mean Squared Error).

### Exercises

#### Flight duration model: Just distance

In [24]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['km'], outputCol='features')
# Consolidate predictor columns
flights = assembler.transform(flights.drop('features'))
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)

In [25]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

23/05/02 19:34:32 WARN Instrumentation: [807997f2] regParam is zero, which might cause numerical instability and overfitting.


+--------+------------------+
|duration|prediction        |
+--------+------------------+
|230     |238.6544157338315 |
|379     |345.6385472841618 |
|240     |213.2901665693866 |
|255     |213.2901665693866 |
|170     |152.11311186827768|
+--------+------------------+
only showing top 5 rows



17.062726863631564

#### Interpreting the coefficients

The linear regression model for flight duration as a function of distance takes the form

$duration = \alpha + \beta \times distance$

where

* $\alpha$ — intercept (component of duration which does not depend on distance) and
* $\beta$ — coefficient (rate at which duration increases as a function of distance; also called the slope).

By looking at the coefficients of your model you will be able to infer

* how much of the average flight duration is actually spent on the ground and
* what the average speed is during a flight.

Instructions

* What's the intercept?
* What are the coefficients? This is a vector.
* Extract the element from the vector which corresponds to the slope for distance.
* Find the average speed in km per hour.

In [26]:
# Intercept (average minutes on ground)
intercept = regression.intercept
print('The average flight duration (minutes) spent on the ground, i.e., when distance is 0 km:')
print(f'\tThe {intercept=:.1f} (minutes) is the component of duration (minutes) independent on distance (km)')
print()

# Coefficients
coefficient = regression.coefficients[0]
print('The distance coefficient is the rate at which duration (minutes) increases as function of distance:')
print(f'\tThe {coefficient=:.4f} is the component of duration (minutes) for each distance unit (km)')
print(f'\tThe {coefficient=:.4f} is the average minutes per kilometer')
print(f'\tThe average speed (km/minute) is {1/coefficient=:.4f}')
print(f'\tThe average speed (km/hour) is {60/coefficient=:.4f}')

The average flight duration (minutes) spent on the ground, i.e., when distance is 0 km:
	The intercept=44.1 (minutes) is the component of duration (minutes) independent on distance (km)

The distance coefficient is the rate at which duration (minutes) increases as function of distance:
	The coefficient=0.0757 is the component of duration (minutes) for each distance unit (km)
	The coefficient=0.0757 is the average minutes per kilometer
	The average speed (km/minute) is 1/coefficient=13.2076
	The average speed (km/hour) is 60/coefficient=792.4540


> **The average speed of a commercial jet is around 850 km/hour. But you got that already from the data!**

In [27]:
# Really?
flights.selectExpr('km / (duration/60)').describe().show()

+-------+----------------------+
|summary|(km / (duration / 60))|
+-------+----------------------+
|  count|                258289|
|   mean|     501.3299838474135|
| stddev|    143.79012533366503|
|    min|                  18.0|
|    max|     911.1864406779662|
+-------+----------------------+



#### Flight duration model: Adding origin airport

In [28]:
flights.show(3, truncate=False)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------+
|mon|dom|dow|carrier|org|depart|duration|delay|km    |label|carrier_idx|org_idx|features|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------+
|10 |10 |1  |OO     |ORD|8.18  |51      |27   |253.0 |1    |2.0        |0.0    |[253.0] |
|11 |22 |1  |OO     |ORD|7.17  |127     |-19  |1188.0|0    |2.0        |0.0    |[1188.0]|
|2  |14 |5  |B6     |JFK|21.17 |365     |60   |3618.0|1    |4.0        |2.0    |[3618.0]|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------+
only showing top 3 rows



In [29]:
# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])
# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights = onehot.transform(flights)
# Create an assembler object
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')
# Consolidate predictor columns
flights = assembler.transform(flights.drop('features'))
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)

In [30]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

23/05/02 19:34:35 WARN Instrumentation: [e6e74c38] regParam is zero, which might cause numerical instability and overfitting.


11.01952980081567

#### Interpreting coefficients

Remember that origin airport, org, has eight possible values (ORD, SFO, JFK, LGA, SMF, SJC, TUS and OGG) which have been one-hot encoded to seven dummy variables in org_dummy.

In [31]:
flights.groupby('org', 'org_idx', 'org_dummy').count().orderBy('count', ascending=False).show()

+---+-------+-------------+-----+
|org|org_idx|    org_dummy|count|
+---+-------+-------------+-----+
|ORD|    0.0|(7,[0],[1.0])|98251|
|SFO|    1.0|(7,[1],[1.0])|50011|
|JFK|    2.0|(7,[2],[1.0])|41068|
|LGA|    3.0|(7,[3],[1.0])|25180|
|SMF|    4.0|(7,[4],[1.0])|16381|
|SJC|    5.0|(7,[5],[1.0])|16068|
|TUS|    6.0|(7,[6],[1.0])| 6025|
|OGG|    7.0|    (7,[],[])| 5305|
+---+-------+-------------+-----+



The values for km and org_dummy have been assembled into features, which has eight columns with sparse representation. Column indices in features are as follows:

        0 — km
        1 — ORD
        2 — SFO
        3 — JFK
        4 — LGA
        5 — SMF
        6 — SJC and
        7 — TUS.

Note that OGG does not appear in this list because it is the reference level for the origin airport category.

In this exercise you'll be using the intercept and coefficients attributes to interpret the model.

The coefficients attribute is a list, where the first element indicates how flight duration changes with flight distance.

Instructions

* Find the average speed in km per hour. This will be different to the value that you got earlier because your model is now more sophisticated.
* What's the average time on the ground at OGG?
* What's the average time on the ground at JFK?
* What's the average time on the ground at LGA?

In [32]:
# Average speed in km per hour
print(f'Average speed in km per hour:\n\t{(60 / regression.coefficients[0])=}')

# Average minutes on ground at OGG
print(f'Average minutes on ground at OGG (reference level):\n\t{regression.intercept=}')

# Average minutes on ground at JFK
avg_ground_jfk = intercept + regression.coefficients[3]
print(f'Average minutes on ground at JFK:\n\t{(intercept + regression.coefficients[3])=}')

# Average minutes on ground at LGA
print(f'Average minutes on ground at LGA:\n\t{(intercept + regression.coefficients[4])=}')

Average speed in km per hour:
	(60 / regression.coefficients[0])=807.7485822157474
Average minutes on ground at OGG (reference level):
	regression.intercept=15.895921738780528
Average minutes on ground at JFK:
	(intercept + regression.coefficients[3])=96.6527776065116
Average minutes on ground at LGA:
	(intercept + regression.coefficients[4])=90.86860944814529


> **You're going to spend over an hour on the ground at JFK or LGA but only around 15 minutes at OGG.**

## Bucketing & Engineering

### More feature engineering
Operations on a single column:
* log()
* sqrt()
* pow()

Operations on two columns:
* product
* ratio.

Examples:
* Mass & Height to BIM: $bmi = mass \div height^2$
* Linear density: $linear\_density = mass \div length$
* Area density: $area_density = mass \div length^2$
* Volume density: $volume_density = mass \div length^3$

> Since you only have the length of the vehicles but not their width or height, the length is being used as a proxy for these missing dimensions.

### Exercises

#### Bucket departure time

In [33]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=np.linspace(0,24,9), inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
flights = buckets.transform(flights)
flights.groupby('depart_bucket').count().orderBy('depart_bucket').show()

# Create a one-hot encoder
onehot = OneHotEncoder(inputCol='depart_bucket', outputCol='depart_dummy')

# One-hot encode the bucketed departure times
flights = onehot.fit(flights).transform(flights)
flights.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+-------------+-----+
|depart_bucket|count|
+-------------+-----+
|          0.0|  198|
|          1.0|  694|
|          2.0|45274|
|          3.0|48280|
|          4.0|48838|
|          5.0|48689|
|          6.0|48401|
|          7.0|17915|
+-------------+-----+

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  8.18|          2.0|(7,[2],[1.0])|
|  7.17|          2.0|(7,[2],[1.0])|
| 21.17|          7.0|    (7,[],[])|
| 12.92|          4.0|(7,[4],[1.0])|
| 13.33|          4.0|(7,[4],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



#### Fligh duration model: Adding departure time

In the previous exercise the departure time was bucketed and converted to dummy variables. Now you're going to include those dummy variables in a regression model for flight duration.

The data are in flights. The km, org_dummy and depart_dummy columns have been assembled into features, where km is index 0, org_dummy runs from index 1 to 7 and depart_dummy from index 8 to 14.

In [34]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy'], outputCol='features')
# Consolidate predictor columns
flights = assembler.transform(flights.drop('features'))
flights[['km', 'org_dummy', 'depart_dummy', 'features']].show(5, truncate=False)

+------+-------------+-------------+------------------------------+
|km    |org_dummy    |depart_dummy |features                      |
+------+-------------+-------------+------------------------------+
|253.0 |(7,[0],[1.0])|(7,[2],[1.0])|(15,[0,1,10],[253.0,1.0,1.0]) |
|1188.0|(7,[0],[1.0])|(7,[2],[1.0])|(15,[0,1,10],[1188.0,1.0,1.0])|
|3618.0|(7,[2],[1.0])|(7,[],[])    |(15,[0,3],[3618.0,1.0])       |
|621.0 |(7,[5],[1.0])|(7,[4],[1.0])|(15,[0,6,12],[621.0,1.0,1.0]) |
|1732.0|(7,[3],[1.0])|(7,[4],[1.0])|(15,[0,4,12],[1732.0,1.0,1.0])|
+------+-------------+-------------+------------------------------+
only showing top 5 rows



The data have been split into training and testing sets and a linear regression model, regression, has been built on the training data. Predictions have been made on the testing data and are available as predictions.

In [35]:
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)
# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)
# Create predictions for the testing data
predictions = regression.transform(flights_test)

23/05/02 19:34:37 WARN Instrumentation: [e9f051ae] regParam is zero, which might cause numerical instability and overfitting.


Instructions

* Find the RMSE for predictions on the testing data.
* Find the average time spent on the ground for flights departing from OGG (org reference level) between 21:00 and 24:00 (depart reference level).
* Find the average time spent on the ground for flights departing from OGG (reference level) between 03:00 and 06:00.
* Find the average time spent on the ground for flights departing from JFK (between 03:00 and 06:00.

In [36]:
# both OFF and 21-24h are references levels
flights.select('km', 'org', 'org_dummy', 'depart', 'depart_dummy', 'features').where('org = "OGG" AND depart >=21 and depart<24').show(1, truncate=False)

+-----+---+---------+------+------------+----------------+
|km   |org|org_dummy|depart|depart_dummy|features        |
+-----+---+---------+------+------------+----------------+
|161.0|OGG|(7,[],[])|21.0  |(7,[],[])   |(15,[0],[161.0])|
+-----+---+---------+------+------------+----------------+
only showing top 1 row



In [37]:
flights.select('km', 'org', 'org_dummy', 'depart', 'depart_dummy', 'features').where('org = "OGG" AND depart >=3 and depart <6').show(1, truncate=False)

+---+---+---------+------+------------+--------+
|km |org|org_dummy|depart|depart_dummy|features|
+---+---+---------+------+------------+--------+
+---+---+---------+------+------------+--------+



In [38]:
flights.select('km', 'org', 'org_dummy', 'depart', 'depart_dummy', 'features').where('org = "JFK" AND depart >=3 and depart<6').show(1, truncate=False)

+------+---+-------------+------+-------------+-----------------------------+
|km    |org|org_dummy    |depart|depart_dummy |features                     |
+------+---+-------------+------+-------------+-----------------------------+
|1754.0|JFK|(7,[2],[1.0])|5.75  |(7,[1],[1.0])|(15,[0,3,9],[1754.0,1.0,1.0])|
+------+---+-------------+------+-------------+-----------------------------+
only showing top 1 row



In [39]:
# Find the RMSE on testing data
from pyspark.ml.evaluation import RegressionEvaluator
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 03:00 and 06:00
avg_night_ogg = regression.intercept + regression.coefficients[9]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 03:00 and 06:00
avg_night_jfk = regression.intercept + regression.coefficients[3] + regression.coefficients[9]
print(avg_night_jfk)

The test RMSE is 10.688463762040891
10.154076497469674
10.65109471840926
62.49567196984156


>**Adding departure time resulted in a smaller RMSE.**

## Regularization

### Loss function (revisited)
Linear regression aims to minimise the MSE.
$$MSE = \frac{1}{N} \sum_{i=1}^{N} \left(y_i - \hat{y_i}\right)^2$$

In this lesson we'll be exploring one such approach to feature selection known as "penalized regression".

The basic idea is that the model is penalized, or punished, for having too many coefficients.

Recall that the conventional regression algorithm chooses coefficients to minimize the loss function, which is average of the squared residuals.

A good model will result in low MSE because its predictions will be close to the observed values.

### Loss function with regularization

Add a regularization term which depends on coefficients.
$$MSE = \frac{1}{N} \sum_{i=1}^{N} \left(y_i - \hat{y_i}\right)^2 + \lambda f(\beta)$$

With penalized regression an additional "regularization" or "shrinkage" term is added to the loss function.

**Rather than depending on the data, this term is a function of the model coefficients.**

### Regularization term
An extra *regularization* term is added to the loss function.

The regularization term can be either

* Lasso — proportion to absolute value of the coefficients
* Ridge — square of the coefficients

>There's a subtle distinction between Lasso and Ridge regression. Both will shrink the coefficients of unimportant predictors. However, whereas Ridge will result in those coefficients being close to zero, Lasso will actually force them to zero precisely.

It's also possible to have a blend of Lasso and Ridge regression.

Strength of regularization determined by parameter $\lambda$:
* $\lambda$ = 0 — no regularization (standard regression)
* $\lambda$ = $\infty$ — complete regularization (all coefficients zero)

> **Ideally you want to choose a value for lambda between these two extremes!**

### Cars: Linear regression
Fit a (standard) Linear Regression model to the training data.

        # default elasticNetParam = 0 | default regParam = 0
        regression = LinearRegression(labelCol='consumption').fit(cars_train)

### Cars: Ridge regression

        # alpha = 0 | lambda = 0.1 -> Ridge
        ridge = LinearRegression(labelCol='consumption', elasticNetParam=0, regParam=0.1)

### Cars: Lasso regression
        # alpha = 1 | lambda = 0.1 -> Lasso
        lasso = LinearRegression(labelCol='consumption', elasticNetParam=1, regParam=0.1)

### Exercises

#### Flight duration model: More features!

In [40]:
flights = OneHotEncoder(inputCols=['dow', 'mon'], outputCols=['dow_dummy', 'mon_dummy']).fit(flights).transform(flights)

features = ['km', 'org_dummy', 'depart_dummy', 'dow_dummy', 'mon_dummy']
assembler = VectorAssembler(inputCols=features, outputCol='features')
# Consolidate predictor columns

flights = assembler.transform(flights.drop('features'))
flights[features + ['features']].show(5, truncate=False)

flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)

+------+-------------+-------------+-------------+---------------+--------------------------------------------+
|km    |org_dummy    |depart_dummy |dow_dummy    |mon_dummy      |features                                    |
+------+-------------+-------------+-------------+---------------+--------------------------------------------+
|253.0 |(7,[0],[1.0])|(7,[2],[1.0])|(6,[1],[1.0])|(11,[10],[1.0])|(32,[0,1,10,16,31],[253.0,1.0,1.0,1.0,1.0]) |
|1188.0|(7,[0],[1.0])|(7,[2],[1.0])|(6,[1],[1.0])|(11,[],[])     |(32,[0,1,10,16],[1188.0,1.0,1.0,1.0])       |
|3618.0|(7,[2],[1.0])|(7,[],[])    |(6,[5],[1.0])|(11,[2],[1.0]) |(32,[0,3,20,23],[3618.0,1.0,1.0,1.0])       |
|621.0 |(7,[5],[1.0])|(7,[4],[1.0])|(6,[3],[1.0])|(11,[5],[1.0]) |(32,[0,6,12,18,26],[621.0,1.0,1.0,1.0,1.0]) |
|1732.0|(7,[3],[1.0])|(7,[4],[1.0])|(6,[1],[1.0])|(11,[3],[1.0]) |(32,[0,4,12,16,24],[1732.0,1.0,1.0,1.0,1.0])|
+------+-------------+-------------+-------------+---------------+--------------------------------------

In [41]:
# Fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Make predictions on testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
print(f'{regression.coefficients=}')

23/05/02 19:34:40 WARN Instrumentation: [c3c2fbb1] regParam is zero, which might cause numerical instability and overfitting.


The test RMSE is 10.602802123410276
regression.coefficients=DenseVector([0.0744, 27.4514, 20.2582, 52.0512, 46.1826, 15.3612, 17.5627, 17.6088, -14.8112, 0.2419, 4.1927, 7.1363, 4.823, 8.9569, 9.0253, -0.0644, -0.1053, -0.1307, -0.0945, -0.0609, -0.1116, -1.5643, -1.8322, -1.9545, -3.2953, -3.925, -3.796, -3.8926, -3.926, -3.7033, -2.4839, -0.3535])


>**With all those non-zero coefficients the model is a little hard to interpret!**

#### Flight duration model: Regularization!

In [42]:
# Fit Lasso model (λ = 1, α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
print(f'{regression.coefficients=}')

# Number of zero coefficients
zero_coeff = sum([beta==0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.58447424577635
regression.coefficients=DenseVector([0.0735, 5.6223, 0.0, 29.1973, 22.206, -2.0498, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.048, 1.232, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
Number of coefficients equal to 0: 25


# Ensemble & Pipelines

## Pipeline

### Exercises

#### Flight duration model: Pipeline stages

In [43]:
# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoder(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

#### Flight duration model: Pipeline model

In [44]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

#
flights_train = flights_train.drop(*['org_idx', 'org_dummy', 'dow_dummy', 'features'])
flights_test = flights_test.drop(*['org_idx', 'org_dummy', 'dow_dummy', 'features'])

# Train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

23/05/02 19:34:45 WARN Instrumentation: [1a7dce9e] regParam is zero, which might cause numerical instability and overfitting.


#### SMS spam pipeline
You haven't looked at the SMS data for quite a while. Last time we did the following:

* split the text into tokens
* removed stop words
* applied the hashing trick
* converted the data from counts to IDF and
* trained a logistic regression model.

In [45]:
# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')
# Remove stop words
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='terms')
# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol=remover.getOutputCol(), outputCol="hash", numFeatures=1024)
idf = IDF(inputCol=hasher.getOutputCol(), outputCol="features")
# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression(regParam=0.2)
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

In [46]:
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema)
# Remove punctuation (REGEX provided) and numbers
sms = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
sms = sms.withColumn('text', regexp_replace(sms.text, '\\d+', ' '))
# Merge multiple spaces
sms = sms.withColumn('text', regexp_replace(sms.text, ' +', ' '))
sms_train, sms_test = sms.randomSplit([.8, .2], seed=13)

In [47]:
pipeline = pipeline.fit(sms_train)

In [48]:
prediction = pipeline.transform(sms_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   41|
|    0|       0.0|  948|
|    1|       1.0|  105|
|    0|       1.0|    2|
+-----+----------+-----+



In [49]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP+TN) / (TP+FP+TN+FN)
print('precision = {:.2f}\nrecall    = {:.2f}\naccuracy  = {:.2f}'.format(precision, recall, accuracy))

precision = 0.98
recall    = 0.72
accuracy  = 0.96


## Cross-Validation

### Exercises

#### Cross validating simple flight duration model

In [50]:
# Read data from CSV file
flights = spark.read.csv('flights-larger.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')
# Convert 'mile' to 'km' and drop 'mile' column (1 mile is equivalent to 1.60934 km)
flights = flights.withColumn('km', F.round(F.col('mile') * 1.60934, 0)).drop('mile')
# Assemble predictors into a single column
flights = VectorAssembler(inputCols=['km'], outputCol='features').transform(flights)
# random split
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)
flights.show(3, truncate=False)

+---+---+---+-------+------+---+------+--------+-----+------+--------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|km    |features|
+---+---+---+-------+------+---+------+--------+-----+------+--------+
|10 |10 |1  |OO     |5836  |ORD|8.18  |51      |27   |253.0 |[253.0] |
|1  |4  |1  |OO     |5866  |ORD|15.5  |102     |null |750.0 |[750.0] |
|11 |22 |1  |OO     |6016  |ORD|7.17  |127     |-19  |1188.0|[1188.0]|
+---+---+---+-------+------+---+------+--------+-----+------+--------+
only showing top 3 rows



In [51]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create an empty parameter grid
params = ParamGridBuilder().build()

# Create objects for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# Create a cross validator
cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-validation builds multiple models, the fit() method can take a little while to complete.
cv.avgMetrics

23/05/02 19:34:49 WARN Instrumentation: [26b37c57] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:50 WARN Instrumentation: [aaafa31a] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:51 WARN Instrumentation: [d17075d0] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:52 WARN Instrumentation: [373ea077] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:53 WARN Instrumentation: [ad905417] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:54 WARN Instrumentation: [6ac4fe1a] regParam is zero, which might cause numerical instability and overfitting.


[17.184539498559413]

In [52]:
predictions = cv.transform(flights_test)
# Calculate the RMSE on testing data
print("The test RMSE is", evaluator.evaluate(predictions))

The test RMSE is 17.21204153787274


In [53]:
# Look at the model coefficients
print(f'{cv.bestModel.coefficients}')

[0.07572046955862763]


#### Cross validating flight duration model pipeline

In [54]:
# Read data from CSV file
flights = spark.read.csv('flights-larger.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')
# Convert 'mile' to 'km' and drop 'mile' column (1 mile is equivalent to 1.60934 km)
flights = flights.withColumn('km', F.round(F.col('mile') * 1.60934, 0)).drop('mile')
# random split
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)
flights.show(3, truncate=False)

+---+---+---+-------+------+---+------+--------+-----+------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|km    |
+---+---+---+-------+------+---+------+--------+-----+------+
|10 |10 |1  |OO     |5836  |ORD|8.18  |51      |27   |253.0 |
|1  |4  |1  |OO     |5866  |ORD|15.5  |102     |null |750.0 |
|11 |22 |1  |OO     |6016  |ORD|7.17  |127     |-19  |1188.0|
+---+---+---+-------+------+---+------+--------+-----+------+
only showing top 3 rows



In [55]:
# Create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# Create an one-hot encoder for the indexed org field
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Create a pipeline and cross-validator.
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
          estimatorParamMaps=params,
          evaluator=evaluator)

In [56]:
# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-validation builds multiple models, the fit() method can take a little while to complete.
cv.avgMetrics

23/05/02 19:34:56 WARN Instrumentation: [ad40667d] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:57 WARN Instrumentation: [4ccd9b68] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:58 WARN Instrumentation: [3c57d1ac] regParam is zero, which might cause numerical instability and overfitting.
23/05/02 19:34:59 WARN Instrumentation: [d59c0ff6] regParam is zero, which might cause numerical instability and overfitting.


[11.194191176221066]

In [57]:
predictions = cv.transform(flights_test)
# Calculate the RMSE on testing data
print("The test RMSE is", evaluator.evaluate(predictions))

The test RMSE is 11.26193940811068


In [58]:
# Look at the model coefficients
print(f'{cv.bestModel.stages[3].coefficients}')

[0.07430685724487272,28.181588242547328,20.199347017329146,52.7008360753772,46.804414004997824,15.642700086393011,17.944909276738414,17.81322776623134]


## Grid Search

### Exercises

#### Optimizing flights linear regression

In [59]:
# Create parameter grid
params = ParamGridBuilder()

# Add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(regression.elasticNetParam, [0.0, 0.5, 1.0])

# Build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

Number of models to be tested:  12


In [60]:
# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-validation builds multiple models, the fit() method can take a little while to complete.
print(cv.avgMetrics)

[11.194422127061321, 11.194801268348169, 11.195581136149205, 11.196376011100767, 11.228310723887455, 11.30283992559167, 11.316858436515284, 11.633405913385612, 11.813595128811798, 14.627907755391655, 17.08384459871639, 19.306444641885214]


In [61]:
predictions = cv.transform(flights_test)
# Calculate the RMSE on testing data
print("The test RMSE is", evaluator.evaluate(predictions))

The test RMSE is 11.261880914187932


In [62]:
# Look at the model coefficients
print(f'{cv.bestModel.stages[3].coefficients}')

[0.07429749423740031,28.04384600659602,20.063235485009532,52.56487077738323,46.66261179485449,15.50175124003904,17.804380246734887,17.672763805589963]


#### Dissecting the best flight duration model

In [63]:
# Get the best model from cross validation
best_model = cv.bestModel

# Look at the stages in the best model
print(best_model.stages)

# Get the parameters for the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(flights_test)
print("RMSE =", evaluator.evaluate(predictions))

[StringIndexerModel: uid=StringIndexer_02eea1e4cb4d, handleInvalid=error, OneHotEncoderModel: uid=OneHotEncoder_7f2b5c13c50a, dropLast=true, handleInvalid=error, numInputCols=1, numOutputCols=1, VectorAssembler_a81c0fbd3ab3, LinearRegressionModel: uid=LinearRegression_28115e64c83c, numFeatures=8]
RMSE = 11.261880914187932


#### SMS spam optimised

In [64]:
# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')
# Remove stop words
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='terms')
# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol='terms', outputCol="hash")
idf = IDF(inputCol=hasher.getOutputCol(), outputCol="features")
# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

In [65]:
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema)
# Remove punctuation (REGEX provided) and numbers
sms = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
sms = sms.withColumn('text', regexp_replace(sms.text, '\\d+', ' '))
# Merge multiple spaces
sms = sms.withColumn('text', regexp_replace(sms.text, ' +', ' '))
sms_train, sms_test = sms.randomSplit([.8, .2], seed=13)

In [66]:
sms.show(3)

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Sorry I'll call l...|    0|
|  2|Dont worry I gues...|    0|
|  3| Call FREEPHONE now |    1|
+---+--------------------+-----+
only showing top 3 rows



In [67]:
# Create parameter grid
params = ParamGridBuilder()

# Add grid for hashing trick parameters
params = params.addGrid(hasher.numFeatures, [1024, 4096, 16384]) \
               .addGrid(hasher.binary, [True, False])

# Add grid for logistic regression parameters
params = params.addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0])

# Build parameter grid
params = params.build()

In [68]:
evaluator = BinaryClassificationEvaluator()

In [69]:
# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

In [70]:
# Train and test model on multiple folds of the training data
cv = cv.fit(sms_train)

# NOTE: Since cross-validation builds multiple models, the fit() method can take a little while to complete.
print(cv.avgMetrics)

23/05/02 19:38:42 WARN BlockManager: Asked to remove block broadcast_19355, which does not exist


[0.9865324890539766, 0.981648802385377, 0.974846362098674, 0.9881255550951987, 0.9347915592965327, 0.846992521452588, 0.986124923610425, 0.5, 0.5, 0.9839475275396181, 0.5, 0.5, 0.986254714042037, 0.9812666587073723, 0.9749246174466236, 0.9875280268355644, 0.9435764459100782, 0.8469132192907945, 0.9859391890771846, 0.5, 0.5, 0.9838777537458983, 0.5, 0.5, 0.9907465808449725, 0.9842098712862292, 0.9799245636286591, 0.9925598542614464, 0.9367113620396077, 0.8480220593402071, 0.9928913434374778, 0.5, 0.5, 0.9919898989932652, 0.5, 0.5, 0.9891676424638248, 0.9842240917755323, 0.980111543939256, 0.9912701832283887, 0.9421866899176214, 0.8521307887833119, 0.9920226011282474, 0.5, 0.5, 0.9915162563827339, 0.5, 0.5, 0.9907579900157673, 0.9800806571645444, 0.9759456158547957, 0.9921991434201001, 0.9403864464468331, 0.8513028643973142, 0.9930182472465658, 0.5, 0.5, 0.9928357652151357, 0.5, 0.5, 0.9890666799192832, 0.9814207163558871, 0.9757377935282024, 0.9908248014997479, 0.943105168509246, 0.8517

In [71]:
predictions = cv.transform(sms_test)

In [72]:
evaluator.evaluate(prediction, {evaluator.metricName: 'areaUnderROC'})

0.9908651766402312

In [73]:
evaluator.evaluate(prediction, {evaluator.metricName: 'areaUnderPR'})

0.9548905405733183

In [74]:
multi_evaluator = MulticlassClassificationEvaluator()

In [75]:
multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

0.961576471412704

In [76]:
multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedRecall"})

0.9607664233576643

In [77]:
multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "f1"})

0.9581370530590708

In [78]:
multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "accuracy"})

0.9607664233576643

In [79]:
multi_evaluator.evaluate(prediction)

0.9581370530590708

In [80]:
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   41|
|    0|       0.0|  948|
|    1|       1.0|  105|
|    0|       1.0|    2|
+-----+----------+-----+



In [81]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP+TN) / (TP+FP+TN+FN)
print(f'{precision=}\n{recall=}\n{accuracy=}')

precision=0.9813084112149533
recall=0.7191780821917808
accuracy=0.9607664233576643


## Ensemble

### Random Forest
Random Forest — an ensemble of Decision Trees  
Creating model diversity:
* each tree trained on
* random subset of data
* random subset of features used for splitting at each node

No two trees in the forest should be the same.

### Gradient-Boosted Trees
Iterative boosting algorithm:
1. Build a Decision Tree and add to ensemble.
2. Predict label for each training instance using ensemble.
3. Compare predictions with known labels.
4. Emphasize training instances with incorrect predictions.
5. Return to 1.

Model improves on each iteration.

### Exercises

#### Delayed flights with Gradient-Boosted Trees

In [82]:
flights = flights.withColumn('label', (F.col('delay') >= 15).cast('integer'))
flights = VectorAssembler(inputCols=['mon', 'depart', 'duration'], outputCol='features').transform(flights)
flights.show(3)

+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|label|         features|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
| 10| 10|  1|     OO|  5836|ORD|  8.18|      51|   27| 253.0|    1| [10.0,8.18,51.0]|
|  1|  4|  1|     OO|  5866|ORD|  15.5|     102| null| 750.0| null| [1.0,15.5,102.0]|
| 11| 22|  1|     OO|  6016|ORD|  7.17|     127|  -19|1188.0|    0|[11.0,7.17,127.0]|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
only showing top 3 rows



In [83]:
flights_train, flights_test = flights.randomSplit([.8,.2], seed=43)
flights.show(3, truncate=False)

+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|km    |label|features         |
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
|10 |10 |1  |OO     |5836  |ORD|8.18  |51      |27   |253.0 |1    |[10.0,8.18,51.0] |
|1  |4  |1  |OO     |5866  |ORD|15.5  |102     |null |750.0 |null |[1.0,15.5,102.0] |
|11 |22 |1  |OO     |6016  |ORD|7.17  |127     |-19  |1188.0|0    |[11.0,7.17,127.0]|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------------+
only showing top 3 rows



In [84]:
# Import the classes required
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create model objects and train on training data
tree = DecisionTreeClassifier().fit(flights_train.dropna())
gbt = GBTClassifier().fit(flights_train.dropna())

# Compare AUC on testing data
evaluator = BinaryClassificationEvaluator()
print(evaluator.evaluate(tree.transform(flights_test.dropna())))
print(evaluator.evaluate(gbt.transform(flights_test.dropna())))

# Find the number of trees and the relative importance of features
print(len(gbt.trees))
print(gbt.featureImportances)

0.6130544933172917
0.6801790462031959
20
(3,[0,1,2],[0.3948239175674082,0.33608675436649965,0.26908932806609215])


#### Delayed flights with a Random Forest

In [85]:
# Create a random forest classifier
from pyspark.ml.classification import RandomForestClassifier
forest = RandomForestClassifier()

# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()

# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()

# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

In [86]:
cv = cv.fit(flights_train.dropna())

23/05/02 19:39:12 WARN DAGScheduler: Broadcasting large task binary with size 1371.7 KiB
23/05/02 19:39:13 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/05/02 19:39:14 WARN DAGScheduler: Broadcasting large task binary with size 1331.6 KiB
23/05/02 19:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1503.6 KiB
23/05/02 19:39:22 WARN DAGScheduler: Broadcasting large task binary with size 1400.9 KiB
23/05/02 19:39:22 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/05/02 19:39:23 WARN DAGScheduler: Broadcasting large task binary with size 1193.4 KiB
23/05/02 19:39:27 WARN DAGScheduler: Broadcasting large task binary with size 1400.9 KiB
23/05/02 19:39:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/05/02 19:39:28 WARN DAGScheduler: Broadcasting large task binary with size 1193.4 KiB
23/05/02 19:39:32 WARN DAGScheduler: Broadcasting large task binary with size 1379.5 KiB
23/05/02 19:39:33 WARN DAGSche

#### Evaluating Random Forest

In [87]:
# Average AUC for each parameter combination in grid
print(cv.avgMetrics)

# Average AUC for the best model
print(max(cv.avgMetrics))

# What's the optimal parameter value for maxDepth?
print(cv.bestModel.explainParam('maxDepth'))
# What's the optimal parameter value for featureSubsetStrategy?
print(cv.bestModel.explainParam('featureSubsetStrategy'))

# AUC for best model on testing data
print(evaluator.evaluate(cv.transform(flights_test.dropna())))

[0.6166140495942163, 0.6609892407042208, 0.6829900362212911, 0.6520726854481138, 0.6651830212615231, 0.6801588426954762, 0.6396107709568064, 0.6654136638407119, 0.6830336262810341, 0.6396107709568064, 0.6654136638407119, 0.6830333345648107]
0.6830336262810341
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30]. (default: 5, current: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (de

23/05/02 19:40:54 WARN DAGScheduler: Broadcasting large task binary with size 1145.0 KiB


0.6839736660828811
